In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from scipy.stats import zscore
from sklearn import preprocessing

In [ ]:
def load_dataset_from_github(api_directory_url):
    """
    Given a GitHub API URL pointing to a repository directory, fetches all CSV files
    in that directory and returns a dictionary mapping filenames to pandas DataFrames.
    
    Parameters:
    -----------
    api_directory_url : str
        GitHub API URL for a repository directory, e.g.
        "https://api.github.com/repos/username/repo/contents/path/to/dir"
    
    Returns:
    --------
    dict[str, pandas.DataFrame]
        A dictionary where each key is a CSV filename (e.g., "data.csv")
        and each value is the corresponding DataFrame obtained from reading
        the file's raw URL.
    array of dataframes    
    """
    response = requests.get(api_directory_url)
    response.raise_for_status()  # Raise an error if request failed
    files = response.json()
    
    dataframes = {}
    for file in files:
        name = file.get("name", "")
        if name.endswith(".csv"):
            raw_url = file.get("download_url")
            if raw_url:
                dataframes[name] = pd.read_csv(raw_url)
    ordered_names = [
        '1- mental-illnesses-prevalence.csv',
        '2- burden-disease-from-each-mental-illness(1).csv',
        '3- adult-population-covered-in-primary-data-on-the-prevalence-of-major-depression.csv',
        '4- adult-population-covered-in-primary-data-on-the-prevalence-of-mental-illnesses.csv',
        '5- anxiety-disorders-treatment-gap.csv',
        '6- depressive-symptoms-across-us-population.csv',
        '7- number-of-countries-with-primary-data-on-prevalence-of-mental-illnesses-in-the-global-burden-of-disease-study.csv'
    ]
    dfs = [dfs_dict[name] for name in ordered_names]
    return dataframes, dfs

In [ ]:
# Load CSV files
file_path = "https://api.github.com/repos/AAI500TeamProject/thementalists-project/contents/Dataset/MentalHealth"
dfs_dict, dfs = load_dataset_from_github(file_path)      

In [ ]:
# Drop columns
for df in dfs:
     df.drop(columns ='Code', inplace=True)

#Rename the lengthy columns to make the table compact
for df in dfs:
     df.rename(columns = {
         'Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized': 'Schizophrenia',
          'Depressive disorders (share of population) - Sex: Both - Age: Age-standardized': 'Depression',
          'Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized': 'Anxiety',
          'Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized': 'Bipolar',
          'Eating disorders (share of population) - Sex: Both - Age: Age-standardized' : 'EatingDisorders'
}, inplace = True)

# standardizing column names
for df in dfs:
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# Trim whitespace in string columns
for df in dfs:
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].str.strip()

# Convert data types where appropriate
for df in dfs:
    if 'year' in df.columns:
        df['year'] = pd.to_numeric(df['year'], errors='coerce', downcast='integer')

# Check for and handle missing values
for i, df in enumerate(dfs):
    print(f"\nMissing values in DataFrame {i}:")
    print(df.isnull().sum())

# Preview each DataFrame
for i, df in enumerate(dfs):
    print(f"\nDataFrame {i} Preview:")
    print(df.head())

In [ ]:
for i, df in enumerate(dfs, start=5):
    print(f"{'='*60}")
    print(f"📊 DataFrame {i} Summary")
    print(f"{'='*60}")
    
    # Show basic info
    print(f"Shape: {df.shape}")
    print(f"\nColumns:\n{list(df.columns)}")
    
    # Show missing values neatly
    print(f"\nMissing Values:\n{df.isnull().sum()[df.isnull().sum() > 0].to_string()}")
    
    # Show preview
    print("\nPreview (first 5 rows):")
    display(df.head())  # Jupyter-friendly display

In [ ]:
for df in dfs:
    df_skip = df.iloc[:, 2:]
    df_skip = df_skip.apply(pd.to_numeric, errors='coerce') #while developing, detect some values are string type, so using this to safely convert them into numeric

    for column in df_skip:

        #draw boxplot and kde before analyte to visualize the outliers
        fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(12, 5))

        axes[0][0].set_title(f'Before handled outliers | Mean: {df_skip[column].mean():.6f}')
        sns.boxplot(y=df_skip[column], ax=axes[0][0])
        sns.kdeplot(df_skip[column], ax=axes[1][0])

        #handle outliers using IQR
        Q1 = df_skip[column].quantile(0.25)
        Q3 = df_skip[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_iqr = df_skip[[column]].copy()
        df_iqr[column]= df_skip[column].clip(lower=lower_bound, upper=upper_bound)

        #draw boxplot after handling outliers
        axes[0][1].set_title(f'After IQR | Mean: {df_iqr[column].mean():.6f}')
        sns.boxplot(y=df_iqr[column], ax=axes[0][1])
        sns.kdeplot(df_iqr[column], ax=axes[1][1])

        # Try z-score method to compare results with IQR method
        df_z = df_skip[[column]].copy()
        df_z['z_score'] = zscore(df_z[column]) 
        df_z['capped'] = df_z[column].where(df_z['z_score'].abs() <= 3) #created capped column to cap values smaller or equal to z-score 3 which is 99.7%

        axes[0][2].set_title(f'After Z-Score | Mean: {df_z[column].mean():.6f}')
        sns.boxplot(y=df_z['capped'].dropna(), ax=axes[0][2])
        sns.kdeplot(df_z['capped'].dropna(), ax=axes[1][2])
        
        plt.show()